In [64]:
import numpy as np
from collections import Counter
def getvote(v):
  # The purpose of this function is to produce a vote (majority belief of the
  # individuals with superior knowledge compared with the code) and the
  # difference between the two votes (k). 
  # Here I assume that the individuals with beliefs = 0 do not vote (abstain),
  # which is a natural proposition when viewed from the political science
  # perspective (if one doesn't have an opinion about a certain dimension, 
  # one abstains from the vote).
  # Specifically, the function takes a vector (v) of arbitrary length and
  # produces the majority vote and advantage over the second choice. It only
  # works with two non-neutral (not 0) beliefs. It can be generalized to more
  # values, but further assumption about the rate of code learning (effect of n
  # and k) need to be made.
    v.sort()
    uniqv = list(filter(lambda x: (x > 0), v))
    freq= Counter([uniqv.index(x)+1 if x in uniqv else 0 for x in v ])
    indx_max=max(freq)
    k= max(freq)-min(freq)
    if(len(uniqv)==1):
        # Here we need to eliminate an situation where with only one vote
        # the max == min and thus k would be 0 (no code learning)
        k=max(freq)
    output_ = list((uniqv[indx_max-1],k))
    return output_

In [62]:
# MODEL VARIABLES  ------------------------
iterations = 80  # number of iteration, originally set to 80 in the paper
time = 50  # iterations until equilibrium, a lazy solution to the steady state problem
m = 30  # number of dimensions
n = 50  # number of people

# *vestigal code ####
# p1 <- matrix (0.5, nrow=n)  # speed of socialization for each person
# should introduce additional code to allow for heterogeneity in learning rates
# p2 <- 0.5  # speed of code learning
# end vestigal code ###

p3 = 0  # turnover
p4 = 0  # environmental turbulence


# lists forming the parmeter space for the learning rates
P1_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
P2_list = [0.1, 0.5, 0.9]

In [63]:
# PREPARING THE OUTPUT MATRIX ------
#OUTPUT = list(0, nrow=length(P1_list), ncol=length(P2_list))
OUTPUT = np.zeros((len(P1_list),len(P2_list)))
c_p1=0
scenario=0


In [ ]:
# SIMULATION  -----------------
for p1 in P1_list :
  c_p1 = c_p1 + 1
  c_p2 = 0  # counter for recording columns in the OUTPUT file
  for p2 in P2_list :
    c_p2 = c_p2 + 1
    scenario = scenario + 1
    for i in range(1,iterations) :
      #str.join("\r","Scenario: ", toString(scenario), " out of ", toString(length(P1_list)*length(P2_list)), ", iteration: ", toString(i))
      print("\r"+"Scenario: "+str(scenario)+" out of "+str(len(P1_list)*len(P2_list))+", iteration: "+ str(i))
      
      # Generating starting objects
      external_reality = (2*(np.random.uniform(m, 0, 2)) - 1).astype(int)
      beliefs = np.random.uniform(n*m,-1,2)[n:m]
      org_code = np.zeros(m)

      for t in range(1,time) :
        # turnover =====
        for x3 in range(1,n) :
          if(np.random.uniform(1) < p3):
            beliefs[:x3] = matrix(math.floor(np.random.uniform(m, min=-1, max=2)), nrow=m)

        # environmental turbulence  =====
        for x4 in range(1,m) :
          if(np.random.uniform(1) < p4):
            external_reality[x4] = external_reality[x4]*(-1)

        # socialization ======
        for n_ in range(1,n):
          for m_ in range(1,m) :
            if(np.random.uniform(1) < p1):
              # individuals don't learn if code is 0 [page 74, line 27:28]
              if(org_code[m_] != 0):
                beliefs[(n_, m_)] = org_code[m_]
        # end: socialization

        # code learning ======
        # * choosing the chosen ones (superior knowledge) ######
        # find out the quality of knowledge for the code and individuals

        if(org_code == external_reality):
            knowl_code = 1
        else:
            knowl_code = 0
        knowl_wkrs = np.zeros(n)  # vector with num of correct dimensions

        for person in range(1,n) :
            if(beliefs[person:] == external_reality):
                knowl_wkrs[person] = 1
            else:
                knowl_wkrs[person] = 0
          #knowl_wkrs[person] = sum(beliefs[person,] == external_reality)

        chosen_ones = np.zeros(n)  # individuals smarter than the org code
        # in the beginning pretty much all workers are better than the code
        # which has zero knowledge

        for person in range(1,n) :
          if(knowl_code < knowl_wkrs[person]):
            chosen_ones[person] = 1
 
        # *superior matrix  ######
        if(sum(chosen_ones) > 0):
          knowl_matrix = matrix(0, nrow=sum(chosen_ones), ncol=m)
          p = 1  # counter for the superior people
          for person in range(1,n) :
            if(chosen_ones[person] == 1):
              knowl_matrix[p:] = beliefs[person:]
              p = p + 1
          # *learning #####
          # Setting up the superior group and its knowledge for the code to learn from it
          # As per the paper, org code learns from the majority vote of superior
          # group.
          for dimension in range(1,m) :
            result = getvote(knowl_matrix[(0,dimension)])
            vote = result[1]
            k = result[2]
            if(np.random.uniform(1) > ((1- p2)^k)):
              org_code[dimension] = vote
      # recording results  ======
      # For now a crude but working version.
      if(org_code == external_reality):
        knowl_code = 1
      else:
        knowl_code = 0
      #knowl_code = sum(org_code == external_reality)
      OUTPUT[(c_p1, c_p2)] = OUTPUT[(c_p1, c_p2)] + (knowl_code/m)
OUTPUT = OUTPUT/iterations  # taking the average

In [ ]:
a = np.asarray(OUTPUT)
file_name = "March1991_i_"+str(i)+"_t_"+str(t)+".csv"
print(file_name)
np.savetxt(file_name, a, delimiter="\"")